<h1>North Zone vs South Zone</h1>
<p>Our goal is to use data to find differences between these two city zones</p>

<h2>Install libraries</h2>

In [17]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [19]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from IPython.display import display

from folium.features import DivIcon

print('Libraries imported.')

Libraries imported.


<h2>Access Foursquare API</h2>

In [20]:
CLIENT_ID = 'XRQHFT4J11A3WCH45N02J4YKGSXASPTXPFJCBXQC4QFAIKQ0' # your Foursquare ID
CLIENT_SECRET = 'KJLWIQJOK5SOIILG1YHSZK42RGYYG351LYPGVP3GBCMY4JSW' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 1000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XRQHFT4J11A3WCH45N02J4YKGSXASPTXPFJCBXQC4QFAIKQ0
CLIENT_SECRET:KJLWIQJOK5SOIILG1YHSZK42RGYYG351LYPGVP3GBCMY4JSW


<h2>Define the reference coordinates for our research</h2>

In [21]:
address = 'Osasco, Brazil'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [22]:
print(f"We'll be using as center for Osasco: {latitude},{longitude}")

We'll be using as center for Osasco: -23.5324859,-46.7916801


<h2>Define the venues around our center</h2>
<p>We'll choose a radius of 10 kilometers around the center and query its venues</p>
<p>We'll pick as venues hospitals, police stations, schools, restaurants, drugstores and supermarkets</p>

In [23]:
radius = 10000
# categoryIds for hospitals, police stations, schools, restaurants, drugstores and supermarkets, pet shops, pharmacies, malls, hotels, clothing stores, gas stations, movie theater, music clubs, stadiums, college, night life, recreation, parking, post office, churches, shops

categories = {
   '4bf58dd8d48988d196941735':'Hospital', 
   '4bf58dd8d48988d12e941735':'Police', 
   '4bf58dd8d48988d13b941735':'School', 
   '4d4b7105d754a06374d81259':'Restaurant', 
   '5745c2e4498e11e7bccabdbd':'Drugstore', 
   '52f2ab2ebcbc57f1066b8b46':'Supermarket',
   '4bf58dd8d48988d100951735':'Pet Shop',
   '4bf58dd8d48988d10f951735':'Drugstore',
   '4bf58dd8d48988d1fd941735':'Mall',
   '4bf58dd8d48988d1fa931735':'Hotel',
   '4bf58dd8d48988d103951735':'Shop',
   '4bf58dd8d48988d113951735':'Gas Station',
   '4bf58dd8d48988d17f941735':'Cinema',
   '4bf58dd8d48988d1e5931735':'Music Club',
   '4bf58dd8d48988d184941735':'Stadium',
   '4d4b7105d754a06372d81259':'College',
   '4d4b7105d754a06376d81259':'Nightlife',
   '4d4b7105d754a06377d81259':'Recreation',
   '4c38df4de52ce0d596b336e1':'Parking',
   '4bf58dd8d48988d172941735':'Post Office',
   '4bf58dd8d48988d131941735':'Church',
   '4d4b7105d754a06378d81259':'Shop',
   '4bf58dd8d48988d175941735':'Gym',
   '4bf58dd8d48988d1fe931735':'Bus Station',
   '4bf58dd8d48988d129951735':'Train Station',
   '53fca564498e1a175f32528b':'Taxi Stand'
}

In [24]:
def explore_venues(catId):
    url = f'https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID} \
                                                        &client_secret={CLIENT_SECRET} \
                                                        &ll={latitude},{longitude} \
                                                        &oauth_token={ACCESS_TOKEN} \
                                                        &v={VERSION} \
                                                        &categoryId={catId} \
                                                        &radius={radius} \
                                                        &limit={LIMIT}'
    results = requests.get(url).json()
    return results

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    print(categories_list)
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
# function that extracts the category name of the venue based on the new classification
def get_category_name(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    print(categories_list)
    print(categories[categories_list[0]['id']])
        
    if len(categories_list) == 0:
        return None
    else:
        return categories[categories_list[0]['id']]

In [27]:
# pick an initial query to form dataframe
results = explore_venues('4bf58dd8d48988d196941735')

# assign relevant part of JSON to venues
venues = results['response']['groups'][0]['items']

# tranform venues into a dataframe
df = pd.json_normalize(venues)

df.drop(df.index, inplace=True)

for id in categories.keys():
    print(id)
    results = explore_venues(id)
    # assign relevant part of JSON to venues
    venues = results['response']['groups'][0]['items']

    # tranform venues into a dataframe
    df1 = pd.json_normalize(venues)
    df1['venue.categories'] = categories[id]
    df = pd.concat([df, df1], ignore_index = True)

4bf58dd8d48988d196941735
4bf58dd8d48988d12e941735
4bf58dd8d48988d13b941735
4d4b7105d754a06374d81259
5745c2e4498e11e7bccabdbd
52f2ab2ebcbc57f1066b8b46
4bf58dd8d48988d100951735
4bf58dd8d48988d10f951735
4bf58dd8d48988d1fd941735
4bf58dd8d48988d1fa931735
4bf58dd8d48988d103951735
4bf58dd8d48988d113951735
4bf58dd8d48988d17f941735
4bf58dd8d48988d1e5931735
4bf58dd8d48988d184941735
4d4b7105d754a06372d81259
4d4b7105d754a06376d81259
4d4b7105d754a06377d81259
4c38df4de52ce0d596b336e1
4bf58dd8d48988d172941735
4bf58dd8d48988d131941735
4d4b7105d754a06378d81259
4bf58dd8d48988d175941735
4bf58dd8d48988d1fe931735
4bf58dd8d48988d129951735
53fca564498e1a175f32528b


In [28]:
# removing the entries that are not in Osasco area and removing duplicated values
df = df[df['venue.location.city'] == 'Osasco']
df

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.neighborhood,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet,venue.venuePage.id,venue.events.count,venue.events.summary
0,e-0-4cb5fc4a55af224b3e99967e-0,0,"[{'summary': 'This spot is popular', 'type': '...",4cb5fc4a55af224b3e99967e,Hospital de Medicina dos Olhos (HMO),"R. Salem Bechara, 297",-23.540037,-46.777470,"[{'label': 'display', 'lat': -23.5400374975879...",1676,06018-180,BR,Centro,Osasco,SP,Brasil,"[R. Salem Bechara, 297, Osasco, SP, 06018-180,...",Hospital,0,[],NaN,NaN,NaN,NaN
1,e-0-4bb09073f964a5203b4c3ce3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4bb09073f964a5203b4c3ce3,Hospital e Maternidade Sino Brasileiro,"Av. Mal. Rondon, 299",-23.531933,-46.782065,"[{'label': 'display', 'lat': -23.5319327819235...",983,06093-020,BR,NaN,Osasco,SP,Brasil,"[Av. Mal. Rondon, 299, Osasco, SP, 06093-020, ...",Hospital,0,[],NaN,NaN,NaN,NaN
2,e-0-4bdf0cb50d69b713e90f66d3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4bdf0cb50d69b713e90f66d3,Hospital Cruzeiro do Sul,"Av. dos Autonomistas, 2502",-23.537705,-46.776992,"[{'label': 'display', 'lat': -23.5377045888326...",1607,06090-010,BR,NaN,Osasco,SP,Brasil,"[Av. dos Autonomistas, 2502, Osasco, SP, 06090...",Hospital,0,[],NaN,NaN,NaN,NaN
3,e-0-4c30707c16adc928d326bf9c-3,0,"[{'summary': 'This spot is popular', 'type': '...",4c30707c16adc928d326bf9c,Hospital Regional de Osasco,R. Ari Barroso,-23.525054,-46.771331,"[{'label': 'display', 'lat': -23.5250538795842...",2235,NaN,BR,NaN,Osasco,SP,Brasil,"[R. Ari Barroso, Osasco, SP, Brasil]",Hospital,0,[],NaN,NaN,NaN,NaN
6,e-0-50f3fc9be4b0482965400d37-6,0,"[{'summary': 'This spot is popular', 'type': '...",50f3fc9be4b0482965400d37,MED CENTER,Rua Madre Roselo,-23.535792,-46.782271,"[{'label': 'display', 'lat': -23.5357918361461...",1028,NaN,BR,NaN,Osasco,SP,Brasil,"[Rua Madre Roselo, Osasco, SP, Brasil]",Hospital,0,[],NaN,NaN,NaN,NaN
8,e-0-4e4f92afe4cdc76dc9b270c8-8,0,"[{'summary': 'This spot is popular', 'type': '...",4e4f92afe4cdc76dc9b270c8,Hospital Antônio Giglio,"R. Pedro Fioreti, 28",-23.532185,-46.777561,"[{'label': 'display', 'lat': -23.5321847869983...",1441,06010-130,BR,NaN,Osasco,SP,Brasil,"[R. Pedro Fioreti, 28, Osasco, SP, 06010-130, ...",Hospital,0,[],NaN,NaN,NaN,NaN
9,e-0-4c963106f6c8ef3b8ace80cf-9,0,"[{'summary': 'This spot is popular', 'type': '...",4c963106f6c8ef3b8ace80cf,Hospital Municipal Antônio Giglio,"R. Pedro Fioretti, 48",-23.532382,-46.776731,"[{'label': 'display', 'lat': -23.5323820994043...",1525,06013-080,BR,Centro,Osasco,SP,Brasil,"[R. Pedro Fioretti, 48, Osasco, SP, 06013-080,...",Hospital,0,[],NaN,NaN,NaN,NaN
10,e-0-4e7349c362e1a77aacba9a8a-10,0,"[{'summary': 'This spot is popular', 'type': '...",4e7349c362e1a77aacba9a8a,"GTO Clinica Fisioterapia, Rpg E Acupuntura",NaN,-23.536269,-46.775363,"[{'label': 'display', 'lat': -23.5362693231507...",1717,NaN,BR,NaN,Osasco,SP,Brasil,"[Osasco, SP, Brasil]",Hospital,0,[],NaN,NaN,NaN,NaN
14,e-0-508fb3d9e4b061e9d61c081d-14,0,"[{'summary': 'This spot is popular', 'type': '...",508fb3d9e4b061e9d61c081d,Fundação Pró-Sangue,Rua Ari Barroso,-23.524875,-46.771647,"[{'label': 'display', 'lat': -23.5248754504386...",2213,NaN,BR,NaN,Osasco,SP,Brasil,"[Rua Ari Barroso, Osasco, SP, Brasil]",Hospital,0,[],NaN,NaN,NaN,NaN
15,e-0-4cc3fd33306e224b51909a6c-15,0,"[{'summary': 'This spot is popular', 'type': '...",4cc3fd33306e224b51909a6c,Hosp. Mat. Amador Aguiar,"Av. Getulio Vargas, 1400",-23.507033,-46.788416,"[{'label': 'display', 'lat': -23.5070326532365...",2852,NaN,BR,NaN,Osasco,SP,Brasil,"[Av. Getulio Vargas, 1400, Osasco, SP, Brasil]",Hospital,0,[],NaN,NaN,NaN,NaN


<h2>Get relevant part of JSON and transform it into a pandas dataframe</h2>

In [29]:
df.shape

(321, 24)

In [30]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_filtered = df.loc[:,filtered_columns]
column_names = ['name', 'category', 'lat', 'lng']
df_filtered.columns = column_names
df_filtered.head()

,name,category,lat,lng
0,Hospital de Medicina dos Olhos (HMO),Hospital,-23.540037,-46.777470
1,Hospital e Maternidade Sino Brasileiro,Hospital,-23.531933,-46.782065
2,Hospital Cruzeiro do Sul,Hospital,-23.537705,-46.776992
3,Hospital Regional de Osasco,Hospital,-23.525054,-46.771331
6,MED CENTER,Hospital,-23.535792,-46.782271


<h2>Display our venues on a map</h2>

In [31]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.Circle(
    [latitude, longitude],
    radius=5000,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.category):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

<h2>Cluster the venues into zones using K-Means</h2>

In [32]:
# set number of clusters
kclusters = 10

clustering = df_filtered.drop(['name', 'category'], axis=1)
clustering

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:1] 

# find the clusters' centers
kmeans.cluster_centers_

array([[-23.53086734, -46.77906345],
       [-23.54649007, -46.76470476],
       [-23.55670021, -46.78537603],
       [-23.5049319 , -46.78994723],
       [-23.53338469, -46.79439459],
       [-23.50667312, -46.77451232],
       [-23.53859134, -46.77286494],
       [-23.52385762, -46.76562126],
       [-23.52410953, -46.81056114],
       [-23.56183824, -46.80326862]])

<h2>Creating a table that contains clusters centers and their labels</h2>

In [33]:
clustering.insert(0, 'Cluster Labels', kmeans.labels_)

In [34]:
clustering.head()

,Cluster Labels,lat,lng
0,6,-23.540037,-46.777470
1,0,-23.531933,-46.782065
2,6,-23.537705,-46.776992
3,7,-23.525054,-46.771331
6,0,-23.535792,-46.782271


<h2>Displaying the venue clusters in the map</h2>

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster, category in zip(clustering['lat'], clustering['lng'], clustering['Cluster Labels'], df_filtered['category']):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=f"Cluster: {cluster}, Category: {category}",
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2>Getting a dataframe with the venue distribution on each cluster</h2>

In [37]:
clustering['category'] = df_filtered['category']

In [38]:
clustering.head()

,Cluster Labels,lat,lng,category
0,6,-23.540037,-46.777470,Hospital
1,0,-23.531933,-46.782065,Hospital
2,6,-23.537705,-46.776992,Hospital
3,7,-23.525054,-46.771331,Hospital
6,0,-23.535792,-46.782271,Hospital


In [39]:
cats = clustering['category'].unique()

In [40]:
clustering['category'].value_counts()

Restaurant       39
Nightlife        34
Supermarket      26
Shop             23
Gas Station      21
Drugstore        19
Hospital         16
School           14
Church           14
Gym              13
Post Office      13
Recreation       12
College          11
Police           11
Mall             11
Pet Shop          9
Hotel             8
Bus Station       6
Parking           5
Stadium           5
Train Station     5
Cinema            3
Music Club        2
Taxi Stand        1
Name: category, dtype: int64

In [41]:
clustering['Cluster Labels'].unique()

array([6, 0, 7, 3, 1, 4, 2, 9, 5, 8], dtype=int32)

In [42]:
clusters = np.sort(clustering['Cluster Labels'].unique())
clusters

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

In [43]:
a = [x for x in clusters]
df_total = pd.DataFrame(columns=a, index=cats)
df_total.head()

,0,1,2,3,4,5,6,7,8,9
Hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Police,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Drugstore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
for index, row in df_total.iterrows():
    for col in row:
        for i in range(10):
            df_total[i][index] = clustering[(clustering['category']==index) & (clustering['Cluster Labels']==i)]['category'].count()
df_total

,0,1,2,3,4,5,6,7,8,9
Hospital,4,1,0,1,0,0,8,2,0,0
Police,3,0,1,2,2,0,2,1,0,0
School,2,3,1,4,2,0,1,1,0,0
Restaurant,3,11,3,0,4,2,9,6,0,1
Drugstore,4,1,4,1,1,0,5,2,0,1
Supermarket,1,2,3,3,3,3,5,1,3,2
Pet Shop,0,0,2,0,0,2,3,0,1,1
Mall,5,2,1,0,0,1,2,0,0,0
Hotel,2,0,0,0,0,0,5,1,0,0
Shop,9,4,2,0,0,1,7,0,0,0


In [45]:
for i in range(10):    
    total = df_total[i].sum()
    print(f'Cluster {i}: {total}')

Cluster 0: 48
Cluster 1: 43
Cluster 2: 30
Cluster 3: 22
Cluster 4: 42
Cluster 5: 12
Cluster 6: 76
Cluster 7: 26
Cluster 8: 10
Cluster 9: 12


<h2>Obtaining the top 10 venue types for each cluster</h2>

In [46]:
df_top10 = pd.DataFrame()
for i in range(10):
    print(f'Cluster {i}:')
    df_cluster = df_total[i].sort_values(ascending = False).head(10)
    aux = pd.DataFrame(data=df_cluster)
    df_top10[i] = aux.index
    display(aux)

    

Cluster 0:


,0
Shop,9
Mall,5
Hospital,4
Drugstore,4
Police,3
Restaurant,3
Gas Station,3
School,2
Church,2
Post Office,2


Cluster 1:


,1
Restaurant,11
Nightlife,4
Shop,4
Recreation,4
College,3
Gym,3
School,3
Supermarket,2
Gas Station,2
Mall,2


Cluster 2:


...

,7
Restaurant,6
Hospital,2
Stadium,2
Drugstore,2
Gas Station,2
Nightlife,2
Parking,1
School,1
College,1
Post Office,1


Cluster 8:


,8
Supermarket,3
Train Station,2
Bus Station,1
Gym,1
Post Office,1
Pet Shop,1
Gas Station,1
Taxi Stand,0
Hotel,0
Police,0


Cluster 9:


,9
Church,3
Post Office,2
Supermarket,2
Restaurant,1
Drugstore,1
College,1
Pet Shop,1
Gas Station,1
Taxi Stand,0
Shop,0


In [47]:
top_10_columns = [
    '1st Most Common Venue',
    '2nd Most Common Venue',
    '3rd Most Common Venue',
    '4th Most Common Venue',
    '5th Most Common Venue',
    '6th Most Common Venue',
    '7th Most Common Venue',
    '8th Most Common Venue',
    '9th Most Common Venue',
    '10th Most Common Venue',
]
df_top10 = df_top10.transpose()
df_top10.columns = top_10_columns
df_top10

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Shop,Mall,Hospital,Drugstore,Police,Restaurant,Gas Station,School,Church,Post Office
1,Restaurant,Nightlife,Shop,Recreation,College,Gym,School,Supermarket,Gas Station,Mall
2,Drugstore,Gas Station,Gym,Church,Restaurant,Supermarket,Recreation,Pet Shop,Shop,Police
3,School,Bus Station,Nightlife,Supermarket,Police,Post Office,Hospital,Gym,Drugstore,Stadium
4,Nightlife,Restaurant,Recreation,Gas Station,Supermarket,Post Office,School,College,Police,Parking
5,Supermarket,Restaurant,Pet Shop,Shop,Church,Recreation,Stadium,Mall,Taxi Stand,Police
6,Restaurant,Hospital,Shop,Nightlife,Drugstore,Supermarket,Hotel,Gas Station,Gym,Church
7,Restaurant,Hospital,Stadium,Drugstore,Gas Station,Nightlife,Parking,School,College,Post Office
8,Supermarket,Train Station,Bus Station,Gym,Post Office,Pet Shop,Gas Station,Taxi Stand,Hotel,Police
9,Church,Post Office,Supermarket,Restaurant,Drugstore,College,Pet Shop,Gas Station,Taxi Stand,Shop


<h2>Display some interesting venue distributions</h2>

In [48]:
def display_map(data, feature):
    df = data.loc[feature]
    top = df.max()

    # create map
    map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for cluster, center in zip(clusters, kmeans.cluster_centers_):
        radius_=30*df[cluster]/top
        if (radius_ != 0):
            circle = folium.CircleMarker(
                    [center[0], center[1]],
                    radius=radius_,
                    popup=f"Cluster {cluster}: {radius_}%",
                    color=rainbow[cluster-1],
                    fill=True,
                    fill_color=rainbow[cluster-1],
                    fill_opacity=0.7)
            circle.add_to(map_clusters)

            folium.Marker([center[0], center[1]], icon=DivIcon(
            icon_size=(-10,-10),
            icon_anchor=(7,20),
            html=f'<div style="font-size: 18pt; color : black">{df[cluster]}</div>',
            )).add_to(map_clusters)
    
    loc = feature
    title_html = '''
                 <h3 align="center" style="font-size:20px"><b>{}</b></h3>
                 '''.format(loc)   

    

    map_clusters.get_root().html.add_child(folium.Element(title_html))

    map_clusters.save('map-with-title.html')

    return map_clusters

In [49]:
display_map(df_total, 'Hospital')

In [50]:
display_map(df_total, 'Church')

In [51]:
display_map(df_total, 'Restaurant')

In [52]:
display_map(df_total, 'Nightlife')

In [53]:
display_map(df_total, 'Shop')

In [54]:
display_map(df_total, 'Supermarket')

In [55]:
display_map(df_total, 'Gym')

In [56]:
display_map(df_total, 'Drugstore')

In [57]:
display_map(df_total, 'School')

<h2>North Zone vs South Zone</h2>

<h3>We'll now find out the differences between South Zone and North Zone with regards to venue distribution</h3>

In [62]:
col_div = [
    'S','S','S','N','S','N','S','N','S','S'
]
df_div = df_total
df_div.columns = col_div
df_div

,S,S,S,N,S,N,S,N,S,S
Hospital,4,1,0,1,0,0,8,2,0,0
Police,3,0,1,2,2,0,2,1,0,0
School,2,3,1,4,2,0,1,1,0,0
Restaurant,3,11,3,0,4,2,9,6,0,1
Drugstore,4,1,4,1,1,0,5,2,0,1
Supermarket,1,2,3,3,3,3,5,1,3,2
Pet Shop,0,0,2,0,0,2,3,0,1,1
Mall,5,2,1,0,0,1,2,0,0,0
Hotel,2,0,0,0,0,0,5,1,0,0
Shop,9,4,2,0,0,1,7,0,0,0


In [63]:
df_north = df_div['N'].sum(axis=1).sort_values(ascending = False).head(10)
df_north

Restaurant     8.0
Supermarket    7.0
Nightlife      5.0
School         5.0
Stadium        4.0
Post Office    3.0
Police         3.0
Gas Station    3.0
Bus Station    3.0
Drugstore      3.0
dtype: float64

In [64]:
df_south = df_div['S'].sum(axis=1).sort_values(ascending = False).head(10)
df_south

Restaurant     31.0
Nightlife      29.0
Shop           22.0
Supermarket    19.0
Gas Station    18.0
Drugstore      16.0
Hospital       13.0
Church         12.0
Recreation     11.0
Gym            11.0
dtype: float64

In [65]:
df_top10_ns = pd.DataFrame()
df_top10_ns['North'] = df_north.index
df_top10_ns['South'] = df_south.index
df_top10_ns = df_top10_ns.transpose()
df_top10_ns.columns = top_10_columns
df_top10_ns

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
North,Restaurant,Supermarket,Nightlife,School,Stadium,Post Office,Police,Gas Station,Bus Station,Drugstore
South,Restaurant,Nightlife,Shop,Supermarket,Gas Station,Drugstore,Hospital,Church,Recreation,Gym
